In [1]:
!conda install -c conda-forge eigen; conda update -n base conda; conda install -c anaconda boost

Solving environment: ...working... done

## Package Plan ##

  environment location: /opt/anaconda3/envs/xeus-cling

  added / updated specs:
    - eigen


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1h             |       haf1e3a3_0         1.9 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.9 MB

The following packages will be UPDATED:

  certifi                anaconda::certifi-2020.6.20-py37_0 --> conda-forge::certifi-2020.6.20-py37h2987424_2

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    anaconda::ca-certificates-2020.10.14-0 --> conda-forge::ca-certificates-2020.6.20-hecda079_0
  openssl                                          anaconda --> conda-forge


Proceed ([y]/n)? 

openssl-1.1.1h       | 1.9 MB    | ########## | 100% 
P

In [2]:
#pragma cling add_include_path("/Users/Thomas/Desktop/StochSimCode/2020_CDT_RandomSys_Oxford/StochasticSimulation/Practical1C++/eigen-3.3.8/Eigen/")
#pragma cling add_include_path("/Users/Thomas/Desktop/StochSimCode/2020_CDT_RandomSys_Oxford/StochasticSimulation/Practical1C++/boost_1_74_0/boost/")

In [3]:
#include <iostream>
#include <math.h>
#include <Eigen>
#include <chrono>
#include <random>
#include <boost/math/special_functions/erf.hpp>

using namespace Eigen;
using namespace std;

In [4]:
int main()
{
    std::random_device rd;  //Will be used to obtain a seed for the random number engine
    std::mt19937 gen(rd()); //Standard mersenne_twister_engine seeded with rd()
    std::uniform_real_distribution<> dis(0.0, 1.0);
    int n=1;

    Matrix2d A; // A corresponds to the matrix Sigma in question 1
    A << 4,1,1,4;
    std::cout << "The matrix A is" << std::endl << A << std::endl;

    LLT<Matrix2d> lltOfA(A); // compute the Cholesky decomposition of A
    Matrix2d L = lltOfA.matrixL(); // retrieve factor L  in the decomposition

    std::cout << "The Cholesky factor L is" << std::endl << L << std::endl;
    std::cout << "To check this, let us compute L * L.transpose()" << std::endl;
    std::cout << L * L.transpose() << std::endl;
    std::cout << "This should equal the matrix A" << std::endl;

    std::chrono::steady_clock::time_point start = std::chrono::steady_clock::now();
    int counter=0;
    while(std::chrono::steady_clock::now() - start <= std::chrono::seconds(60)){
        MatrixXf w(2,n);
        for(int i=0; i<n; i++){
            Vector2d tempMat;
            tempMat << -sqrt(2) * boost::math::erfc_inv(2*dis(gen)), -sqrt(2) * boost::math::erfc_inv(2*dis(gen));
            Vector2d tempVec;
            tempVec << L*tempMat;
            w(0,i) = tempVec(0,0);
            w(1, i) = tempVec(1, 0);
            // To check we are indeed generating normals
            //double temp = dis(gen);
            //cout << temp << endl;
            //cout << ICDFnormal(temp) << endl;
            //cout << 0.5 * boost::math::erfc(-(ICDFnormal(temp))/(sqrt(2))) << endl;
        }
        counter+=1;
    }
    std::cout << "We have generated " + std::to_string(counter) + " normals with covariance A."<< std::endl;
}

In [ ]:
main();

The matrix A is
4 1
1 4
The Cholesky factor L is
      2       0
    0.5 1.93649
To check this, let us compute L * L.transpose()
4 1
1 4
This should equal the matrix A
